In [ ]:
import pandas as pd
from matplotlib.pyplot import figure
from scipy.stats import entropy
import utils
import numpy as np

In [ ]:
#---------------------------------------------------
#
# Config *** TO UPDATE ***
#
#---------------------------------------------------

# *** TO UPDATE: change the team number with your own team number
TEAM_NUMBER = "TeamDemo"

# File with AOIS definitions 
AOIS_DEFINITION_FILE = "raw-data/"+TEAM_NUMBER+"/aois definitions/aois.csv"

# This file should be automatically generated from "5. AOI Visits"
DWELLS_FILE = "data/dwells.csv"

In [ ]:
#----------------------------------------------------------------------------------------
#
#  1. Transition Matrix and Markov Model
#  
#
#----------------------------------------------------------------------------------------

In [ ]:
#----------------------------------------------------------------------------------------
#Assumption: the AOIs are not overlapping
#----------------------------------------------------------------------------------------

In [ ]:
# Read aois definition
aois_df = pd.read_csv(AOIS_DEFINITION_FILE)

In [ ]:
# Display aois definition
display(aois_df)

In [ ]:
# Read dwell data using pandas library
data = pd.read_csv(DWELLS_FILE)

# set display.max_columns to none, to show all the columns when using head()
pd.set_option('display.max_columns', None)

In [ ]:
# Preview dwells data
data.head()

In [ ]:
#----------------------------------------------------------------------------------------
#
# 1.1 Identify transitions and generate both transition matrix and markov model (including self-transitions e.g., transitions from AOI1 to AOI1)
#
#----------------------------------------------------------------------------------------

In [ ]:
for task in data['SourceStimuliName'].unique():
    
    print(f"\n Transition Matrix for task: {task} \n")
    
    #Apply filtering
    taskData = data[data['SourceStimuliName']==task].copy(deep=True)
    #Reset data frame index
    taskData = taskData.reset_index(drop=True)

    # get list of AOIs
    AoisToConsider = aois_df[aois_df['task']==task]["AOI"].copy(deep=True).tolist()

    #Keep only aois within AoisToConsider
    taskData = taskData[taskData['VisitedAOI'].isin(AoisToConsider)].copy(deep=True)
    
    transitions = pd.DataFrame()
    transitions[['Respondent','SourceStimuliName','TransitionSource','TransitionTarget']] = taskData.apply(lambda x: utils.identifyTransition(taskData,x.name), axis=1)
    
    # Generate transition matrix
    transitionMatrix = utils.generateTransitionMatrix(transitions,'TransitionSource','TransitionTarget')
    
    print(f"* Transition Matrix:")
    # Display transitionMatrix
    display(transitionMatrix)
    
    # Export transitionMatrix as CSV
    transitionMatrix.to_csv(f"data/transitionMatrix_{task}.csv")
    
    # Generate a markov model i.e., transition matrix with values normalized over each row (by dividing all values by the sum of values in row)
    transitionMatrixNormalized = utils.generateTransitionMatrix(transitions,'TransitionSource','TransitionTarget',normalize='index')
    
    # Round the values in transitionMatrixNormalized to three decimal places
    transitionMatrixNormalized = transitionMatrixNormalized.round(3)
    
    print(f"* Markov Model:")
    # Display transitionMatrixNormalized
    display(transitionMatrixNormalized)

    # Export transitionMatrix as CSV
    transitionMatrixNormalized.to_csv(f"data/transitionMatrixNormalized-MarkovModel_{task}.csv")



In [ ]:
#----------------------------------------------------------------------------------------
#
# 1.2 Identify transitions and generate both transition matrix and markov model (with no self-transitions)
#
#----------------------------------------------------------------------------------------

In [ ]:
# drop consecutive duplicate VisitedAOIs
dataNoConsDupVisits = data[['VisitedAOI']]     
dataNoConsDupVisits = data[(dataNoConsDupVisits.ne(dataNoConsDupVisits.shift())).any(axis=1)]

for task in data['SourceStimuliName'].unique():
    
    print(f"\n Transition Matrix for task: {task} (with no self-transitions)\n")
    
    #Apply filtering
    taskData = dataNoConsDupVisits[dataNoConsDupVisits['SourceStimuliName']==task].copy(deep=True)
    #Reset data frame index
    taskData = taskData.reset_index(drop=True)

    # get list of AOIs
    AoisToConsider = aois_df[aois_df['task']==task]["AOI"].copy(deep=True).tolist()

    #Keep only aois within AoisToConsider
    taskData = taskData[taskData['VisitedAOI'].isin(AoisToConsider)].copy(deep=True)
    
    transitions = pd.DataFrame()
    transitions[['Respondent','SourceStimuliName','TransitionSource','TransitionTarget']] = taskData.apply(lambda x: utils.identifyTransition(taskData,x.name), axis=1)
    
    # Generate transition matrix
    transitionMatrix = utils.generateTransitionMatrix(transitions,'TransitionSource','TransitionTarget')
    
    print(f"* Transition Matrix:")
    # Display transitionMatrix
    display(transitionMatrix)
    
    # Export transitionMatrix as CSV
    transitionMatrix.to_csv(f"data/transitionMatrix_noSelfTransition_{task}.csv")
    
    # Generate a markov model i.e., transition matrix with values normalized over each row (by dividing all values by the sum of values in row)
    transitionMatrixNormalized = utils.generateTransitionMatrix(transitions,'TransitionSource','TransitionTarget',normalize='index')
    
    # Round the values in transitionMatrixNormalized to three decimal places
    transitionMatrixNormalized = transitionMatrixNormalized.round(3)

    print(f"* Markov Model:")
    # Display transitionMatrixNormalized
    display(transitionMatrixNormalized)

    # Export transitionMatrix as CSV
    transitionMatrixNormalized.to_csv(f"data/transitionMatrixNormalized-MarkovModel_noSelfTransition_{task}.csv")
